# MLPy — Sesión 7: Técnicas de Regularización

En esta sesión vamos a **provocar overfitting** a propósito y luego aplicar técnicas de regularización para mejorar la **generalización**.

**Mantendremos constantes** (en la medida de lo posible):
- dataset (clasificación 2D)
- esquema de entrenamiento
- métrica principal (accuracy)
- comparación por curvas train/val


In [ ]:
import math
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

torch.manual_seed(0)


## 1) Dataset 2D con split train/val

Usamos un dataset sintético que permite ver rápidamente:
- separación en el plano (x0, x1)
- overfitting cuando el modelo es demasiado flexible


In [ ]:
# Dataset base (dos nubes)
N = 1200
x0 = torch.randn(N//2, 2) + torch.tensor([-2.0, 0.0])
x1 = torch.randn(N//2, 2) + torch.tensor([ 2.0, 0.0])
X = torch.cat([x0, x1], dim=0)
y = torch.cat([torch.zeros(N//2), torch.ones(N//2)]).long()

# Mezclar
perm = torch.randperm(N)
X, y = X[perm], y[perm]

# Hacemos un train pequeño para inducir overfitting
n_train = 160
X_train, y_train = X[:n_train], y[:n_train]
X_val,   y_val   = X[n_train:], y[n_train:]

train_ds = TensorDataset(X_train, y_train)
val_ds   = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=256, shuffle=False)

X_train.shape, X_val.shape


## 2) Funciones auxiliares (train/eval + curvas)

Incluimos:
- `train_one_epoch(...)`
- `evaluate(...)`
- utilidades para graficar curvas

También creamos una función `fit(...)` que guarda historial.


In [ ]:
criterion = nn.CrossEntropyLoss()

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    loss_sum, correct, n = 0.0, 0, 0
    for xb, yb in loader:
        logits = model(xb)
        loss = criterion(logits, yb)
        loss_sum += loss.item() * xb.size(0)
        pred = logits.argmax(dim=1)
        correct += (pred == yb).sum().item()
        n += xb.size(0)
    return loss_sum / n, correct / n

def train_one_epoch(model, loader, optimizer, l1_lambda=0.0, augment_noise_std=0.0):
    model.train()
    loss_sum, correct, n = 0.0, 0, 0

    for xb, yb in loader:
        # Data augmentation simple: ruido gaussiano en entrenamiento
        if augment_noise_std > 0:
            xb = xb + augment_noise_std * torch.randn_like(xb)

        logits = model(xb)
        loss = criterion(logits, yb)

        # L1 manual (PyTorch optimizers no traen "l1" built-in)
        if l1_lambda > 0:
            l1_pen = 0.0
            for p in model.parameters():
                l1_pen = l1_pen + p.abs().sum()
            loss = loss + l1_lambda * l1_pen

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_sum += loss.item() * xb.size(0)
        pred = logits.argmax(dim=1)
        correct += (pred == yb).sum().item()
        n += xb.size(0)

    return loss_sum / n, correct / n

def plot_history(hist, title):
    epochs = range(1, len(hist["train_loss"]) + 1)
    plt.figure()
    plt.plot(epochs, hist["train_loss"], label="train loss")
    plt.plot(epochs, hist["val_loss"], label="val loss")
    plt.xlabel("epoch")
    plt.ylabel("loss")
    plt.title(title)
    plt.legend()
    plt.show()

    plt.figure()
    plt.plot(epochs, hist["train_acc"], label="train acc")
    plt.plot(epochs, hist["val_acc"], label="val acc")
    plt.xlabel("epoch")
    plt.ylabel("accuracy")
    plt.title(title)
    plt.legend()
    plt.show()

def fit(model, train_loader, val_loader, optimizer, epochs=120,
        l1_lambda=0.0, augment_noise_std=0.0,
        early_stopping=False, patience=12):
    hist = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}
    best_val = float("inf")
    best_state = None
    bad_epochs = 0

    for epoch in range(1, epochs + 1):
        tr_loss, tr_acc = train_one_epoch(
            model, train_loader, optimizer,
            l1_lambda=l1_lambda,
            augment_noise_std=augment_noise_std
        )
        va_loss, va_acc = evaluate(model, val_loader)

        hist["train_loss"].append(tr_loss)
        hist["train_acc"].append(tr_acc)
        hist["val_loss"].append(va_loss)
        hist["val_acc"].append(va_acc)

        # Early stopping por val loss
        if early_stopping:
            if va_loss < best_val - 1e-6:
                best_val = va_loss
                best_state = {k: v.detach().clone() for k, v in model.state_dict().items()}
                bad_epochs = 0
            else:
                bad_epochs += 1
                if bad_epochs >= patience:
                    break

    if early_stopping and best_state is not None:
        model.load_state_dict(best_state)

    return hist


## 3) Un modelo "grande" para inducir overfitting

Con pocos datos de entrenamiento, una red con muchas unidades puede memorizar.

Incluimos una variante con dropout (probabilidad configurable).


In [ ]:
class BigMLP(nn.Module):
    def __init__(self, dropout_p=0.0):
        super().__init__()
        layers = [
            nn.Linear(2, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
        ]
        if dropout_p > 0:
            # insert dropout between blocks (simple strategy)
            layers = [
                nn.Linear(2, 128), nn.ReLU(), nn.Dropout(dropout_p),
                nn.Linear(128, 128), nn.ReLU(), nn.Dropout(dropout_p),
                nn.Linear(128, 128), nn.ReLU(), nn.Dropout(dropout_p),
            ]
        layers.append(nn.Linear(128, 2))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

def run_experiment(name, model, optimizer, **fit_kwargs):
    hist = fit(model, train_loader, val_loader, optimizer, **fit_kwargs)
    val_loss, val_acc = evaluate(model, val_loader)
    print(f"{name} | val_loss={val_loss:.4f} | val_acc={val_acc:.4f} | epochs={len(hist['train_loss'])}")
    plot_history(hist, title=name)
    return hist, (val_loss, val_acc)


## 4) Baseline: sin regularización (espera overfitting)

Entrenamos sin L1/L2, sin dropout, sin early stopping, sin augmentation.


In [ ]:
model_base = BigMLP(dropout_p=0.0)
opt_base = torch.optim.SGD(model_base.parameters(), lr=0.08)

hist_base, metrics_base = run_experiment(
    "Baseline (sin regularización)",
    model_base, opt_base,
    epochs=160
)


## 5) L2 (weight decay)

Implementación práctica: `weight_decay` en el optimizador.

Interpretación: penaliza \(\|W\|_2^2\) y empuja pesos más pequeños.


In [ ]:
model_l2 = BigMLP(dropout_p=0.0)
opt_l2 = torch.optim.SGD(model_l2.parameters(), lr=0.08, weight_decay=1e-3)

hist_l2, metrics_l2 = run_experiment(
    "L2 (weight_decay=1e-3)",
    model_l2, opt_l2,
    epochs=160
)


## 6) L1 (penalización manual)

Aquí implementamos L1 agregando \(\lambda \sum |w|\) al loss.

Nota: L1 puede requerir tuning más cuidadoso que L2.


In [ ]:
model_l1 = BigMLP(dropout_p=0.0)
opt_l1 = torch.optim.SGD(model_l1.parameters(), lr=0.08)

hist_l1, metrics_l1 = run_experiment(
    "L1 (lambda=1e-5)",
    model_l1, opt_l1,
    epochs=160,
    l1_lambda=1e-5
)


## 7) Dropout

Dropout introduce ruido estructurado en la red (apaga neuronas al azar).

Regla práctica:
- comenzar con p=0.1–0.3 en MLPs


In [ ]:
model_do = BigMLP(dropout_p=0.25)
opt_do = torch.optim.SGD(model_do.parameters(), lr=0.08)

hist_do, metrics_do = run_experiment(
    "Dropout (p=0.25)",
    model_do, opt_do,
    epochs=160
)


## 8) Early stopping

Entrenamos muchos epochs, pero detenemos cuando la validación deja de mejorar.

Ventaja: suele ser muy efectivo y barato.


In [ ]:
model_es = BigMLP(dropout_p=0.0)
opt_es = torch.optim.SGD(model_es.parameters(), lr=0.08)

hist_es, metrics_es = run_experiment(
    "Early stopping (patience=12)",
    model_es, opt_es,
    epochs=250,
    early_stopping=True,
    patience=12
)


## 9) Data augmentation simple: ruido en inputs

En este dataset toy, usamos **ruido gaussiano en entrenamiento** como augmentación.

Objetivo: que el modelo sea menos sensible a variaciones pequeñas.


In [ ]:
model_aug = BigMLP(dropout_p=0.0)
opt_aug = torch.optim.SGD(model_aug.parameters(), lr=0.08)

hist_aug, metrics_aug = run_experiment(
    "Augmentación (ruido std=0.15)",
    model_aug, opt_aug,
    epochs=160,
    augment_noise_std=0.15
)


## 10) Resumen rápido de resultados

Comparamos métricas finales de validación.

**Nota:** por el azar (inicialización/mini-batches), los números pueden variar, pero las tendencias deberían mantenerse.


In [ ]:
results = [
    ("Baseline",) + metrics_base,
    ("L2",) + metrics_l2,
    ("L1",) + metrics_l1,
    ("Dropout",) + metrics_do,
    ("Early stopping",) + metrics_es,
    ("Augmentación",) + metrics_aug,
]

for name, vloss, vacc in results:
    print(f"{name:15s}  val_loss={vloss:.4f}  val_acc={vacc:.4f}")


## 11) Preguntas de reflexión

1. ¿Qué técnica te dio la mejor mejora en validación con el menor costo?
2. ¿Qué técnica fue más sensible a hiperparámetros (por ejemplo \(\lambda\))?
3. ¿Qué pasa si combinamos técnicas (p.ej. L2 + early stopping + dropout)?
